# Neural Networks

Read the documentation for training a simple convolutional neural network (ref. http: //www.vlfeat.org/matconvnet/training/). Train a CNN with two convolutional layers and one fully connected layer, with the architecture specified as follows: number of nodes: 20-50-500-21. The number of the nodes in the last layer is fixed as 21 as we are per- forming 21-category (20 CMU PIE faces plus 1 for yourself) classification. Convolutional kernel sizes are set as 5. Each convolutional layer is followed by a max pooling layer with a kernel size of 2 and stride of 2. The fully connected layer is followed by ReLU. Train the network and report the final classification performance.

### Read all the images

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits import mplot3d

In [2]:
def read_images():
    dir='PIE'
    PIE=os.listdir(dir)
    all_img = []
    img_folders=[dir+'/' + photo for photo in PIE]
    selected_subjects = np.random.choice(range(len(img_folders)), 20, replace=False)
    
    selected_sub_folders = [img_folders[index] for index in selected_subjects]
    print('Selected Subjects:')
    print(selected_sub_folders)

    for img_dir in selected_sub_folders:
        img_path=os.listdir(img_dir)
        img_dir_index=[img_dir+'/' + photo for photo in img_path]
        all_img.append(img_dir_index)

    flat_list = []
    for sublist in all_img:
        for item in sublist:
            flat_list.append(item)
    
    images = np.array([plt.imread(image) for image in flat_list], dtype=np.int16)
    labels = np.zeros((len(images)))

    for i, path in enumerate(flat_list):
        labels[i] = path.split('/')[1]
    return images, labels

In [3]:
images, labels = read_images()
n_samples, image_size, _ = images.shape
n_features = image_size * image_size

Selected Subjects:
['PIE/31', 'PIE/7', 'PIE/43', 'PIE/27', 'PIE/53', 'PIE/6', 'PIE/42', 'PIE/23', 'PIE/41', 'PIE/52', 'PIE/38', 'PIE/54', 'PIE/49', 'PIE/16', 'PIE/65', 'PIE/10', 'PIE/2', 'PIE/26', 'PIE/19', 'PIE/44']


In [4]:
np.unique(labels).shape

(20,)

### Randomly select 500 images

In [5]:
def select_images(images, labels, select_num):
    num_images = len(images)
    np.random.seed(99)
    if select_num == len(images):
        return images.reshape([select_num, n_features]), labels
    rand_images_list = np.random.randint(0, num_images, select_num)
    X = images[rand_images_list].reshape([select_num, n_features])
    y = labels[rand_images_list]
    return X, y

In [6]:
X, y = select_images(images, labels, 500)

In [7]:
np.unique(labels).shape

(20,)

## Convolutional Neural Network

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten
from keras import optimizers
import keras.backend as K

Using TensorFlow backend.


In [9]:
X, y = select_images(images, labels, n_samples)

In [10]:
X = X.reshape(-1, image_size, image_size, 1)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [12]:
# binary encode
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False,categories='auto')
y_train_reshaped = y_train.reshape(len(y_train), 1)
y_train_onehot_encoded = onehot_encoder.fit_transform(y_train_reshaped)
y_test_reshaped = y_test.reshape(len(y_test), 1)
y_test_onehot_encoded = onehot_encoder.fit_transform(y_test_reshaped)

In [13]:
K.clear_session()

model = Sequential()

model.add(Conv2D(20, (5, 5), activation='relu', input_shape=(image_size, image_size, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(50, (5, 5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(500, activation='relu'))

model.add(Dense(20, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),  
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [14]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 20)        520       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 50)        25050     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 50)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1250)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               625500    
_________________________________________________________________
dense_2 (Dense)              (None, 20)               

In [15]:
model.fit(X_train, y_train_onehot_encoded, batch_size=128,
          epochs=50, verbose=1, validation_split=0.3)

Instructions for updating:
Use tf.cast instead.
Train on 1662 samples, validate on 713 samples
Epoch 1/50
1662/1662 [==============================] - 5s 3ms/step - loss: 13.4497 - acc: 0.0770 - val_loss: 11.8174 - val_acc: 0.1262
Epoch 2/50
1662/1662 [==============================] - 4s 2ms/step - loss: 10.8211 - acc: 0.1480 - val_loss: 10.5419 - val_acc: 0.1388
Epoch 3/50
1662/1662 [==============================] - 4s 3ms/step - loss: 9.1452 - acc: 0.1877 - val_loss: 8.6931 - val_acc: 0.1809
Epoch 4/50
1662/1662 [==============================] - 4s 3ms/step - loss: 6.8116 - acc: 0.2593 - val_loss: 7.0525 - val_acc: 0.2945
Epoch 5/50
1662/1662 [==============================] - 4s 3ms/step - loss: 5.4837 - acc: 0.3502 - val_loss: 4.5913 - val_acc: 0.3043
Epoch 6/50
1662/1662 [==============================] - 4s 3ms/step - loss: 3.5137 - acc: 0.3959 - val_loss: 3.1911 - val_acc: 0.3661
Epoch 7/50
1662/1662 [==============================] - 4s 2ms/step - loss: 2.3847 - acc: 0.4663 

In [16]:
model.evaluate(X_test, y_test_onehot_encoded)

1019/1019 [==============================] - 1s 1ms/step


[0.1858351557891198, 0.9538763499470525]